# HAR CNN training 

In [377]:
# Imports
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

In [378]:
%run utils/utilities.py

In [379]:
class_label_vn = [u"Cổ tay",u"Cổ chân",u"Bả vai",u"Xoay người",u"Xoay đầu gối",u"Đi bộ",u"Chạy",u"Đạp xe",u"Đánh răng",u"Rửa tay",u"Lau bàn",u"Nạo",u"Thái",u"Trộn",u"Lên cầu thang",u"Xuống cầu thang"]

## Prepare data

In [380]:
rootDatasetDir = "./datasets/PTIT/normalized"
X_train, labels_train, list_ch_train = read_data(data_path="./datasets/PTIT/normalized", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path=rootDatasetDir, split="test") # test
assert list_ch_train == list_ch_test, "Mistmatch in channels!"

x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr
x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr


In [381]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [382]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [383]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [384]:
# Imports
import tensorflow as tf

### Hyperparameters

In [385]:
batch_size = 600       # Batch size
seq_len = WINDOWN_SIZE          # Number of steps or window size
learning_rate = 0.0001
epochs = 2000

n_classes = NUM_CLASS
n_channels = NUM_CHANNEL
print "n_channels %d" % n_channels

n_channels 9


### Construct the graph
Placeholders

In [386]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

In [387]:
with graph.as_default():
    # (batch, 128, 9) --> (batch, 64, 18)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Now, flatten and pass to the classifier

In [388]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_4, (-1, 8*144))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [ ]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

('Epoch: 0/2000', 'Iteration: 5', 'Train loss: 2.858363', 'Train acc: 0.071667')
('Epoch: 1/2000', 'Iteration: 10', 'Train loss: 2.820016', 'Train acc: 0.068333')
('Epoch: 1/2000', 'Iteration: 10', 'Validation loss: 2.760466', 'Validation acc: 0.071667')
('Epoch: 2/2000', 'Iteration: 15', 'Train loss: 2.767467', 'Train acc: 0.076667')
('Epoch: 3/2000', 'Iteration: 20', 'Train loss: 2.712113', 'Train acc: 0.098333')
('Epoch: 3/2000', 'Iteration: 20', 'Validation loss: 2.689046', 'Validation acc: 0.098333')
('Epoch: 4/2000', 'Iteration: 25', 'Train loss: 2.713872', 'Train acc: 0.100000')
('Epoch: 5/2000', 'Iteration: 30', 'Train loss: 2.670680', 'Train acc: 0.101667')
('Epoch: 5/2000', 'Iteration: 30', 'Validation loss: 2.642451', 'Validation acc: 0.095000')
('Epoch: 6/2000', 'Iteration: 35', 'Train loss: 2.668613', 'Train acc: 0.106667')
('Epoch: 7/2000', 'Iteration: 40', 'Train loss: 2.640273', 'Train acc: 0.111667')
('Epoch: 7/2000', 'Iteration: 40', 'Validation loss: 2.610223', 'Vali

('Epoch: 64/2000', 'Iteration: 325', 'Train loss: 1.407504', 'Train acc: 0.553333')
('Epoch: 65/2000', 'Iteration: 330', 'Train loss: 1.373470', 'Train acc: 0.541667')
('Epoch: 65/2000', 'Iteration: 330', 'Validation loss: 1.328496', 'Validation acc: 0.600000')
('Epoch: 66/2000', 'Iteration: 335', 'Train loss: 1.323758', 'Train acc: 0.566667')
('Epoch: 67/2000', 'Iteration: 340', 'Train loss: 1.347815', 'Train acc: 0.555000')
('Epoch: 67/2000', 'Iteration: 340', 'Validation loss: 1.293897', 'Validation acc: 0.605000')
('Epoch: 68/2000', 'Iteration: 345', 'Train loss: 1.294908', 'Train acc: 0.571667')
('Epoch: 69/2000', 'Iteration: 350', 'Train loss: 1.302216', 'Train acc: 0.568333')
('Epoch: 69/2000', 'Iteration: 350', 'Validation loss: 1.260232', 'Validation acc: 0.620000')
('Epoch: 70/2000', 'Iteration: 355', 'Train loss: 1.283090', 'Train acc: 0.571667')
('Epoch: 71/2000', 'Iteration: 360', 'Train loss: 1.260313', 'Train acc: 0.583333')
('Epoch: 71/2000', 'Iteration: 360', 'Validati

('Epoch: 127/2000', 'Iteration: 640', 'Train loss: 0.686708', 'Train acc: 0.771667')
('Epoch: 127/2000', 'Iteration: 640', 'Validation loss: 0.712298', 'Validation acc: 0.786667')
('Epoch: 128/2000', 'Iteration: 645', 'Train loss: 0.692885', 'Train acc: 0.795000')
('Epoch: 129/2000', 'Iteration: 650', 'Train loss: 0.686818', 'Train acc: 0.793333')
('Epoch: 129/2000', 'Iteration: 650', 'Validation loss: 0.703948', 'Validation acc: 0.793333')
('Epoch: 130/2000', 'Iteration: 655', 'Train loss: 0.658683', 'Train acc: 0.803333')
('Epoch: 131/2000', 'Iteration: 660', 'Train loss: 0.683652', 'Train acc: 0.793333')
('Epoch: 131/2000', 'Iteration: 660', 'Validation loss: 0.694215', 'Validation acc: 0.795000')
('Epoch: 132/2000', 'Iteration: 665', 'Train loss: 0.649913', 'Train acc: 0.801667')
('Epoch: 133/2000', 'Iteration: 670', 'Train loss: 0.650563', 'Train acc: 0.806667')
('Epoch: 133/2000', 'Iteration: 670', 'Validation loss: 0.686871', 'Validation acc: 0.795000')
('Epoch: 134/2000', 'Iter

('Epoch: 189/2000', 'Iteration: 950', 'Train loss: 0.452441', 'Train acc: 0.870000')
('Epoch: 189/2000', 'Iteration: 950', 'Validation loss: 0.540290', 'Validation acc: 0.835000')
('Epoch: 190/2000', 'Iteration: 955', 'Train loss: 0.451448', 'Train acc: 0.858333')
('Epoch: 191/2000', 'Iteration: 960', 'Train loss: 0.439700', 'Train acc: 0.881667')
('Epoch: 191/2000', 'Iteration: 960', 'Validation loss: 0.535441', 'Validation acc: 0.831667')
('Epoch: 192/2000', 'Iteration: 965', 'Train loss: 0.417836', 'Train acc: 0.886667')
('Epoch: 193/2000', 'Iteration: 970', 'Train loss: 0.426814', 'Train acc: 0.878333')
('Epoch: 193/2000', 'Iteration: 970', 'Validation loss: 0.531974', 'Validation acc: 0.836667')
('Epoch: 194/2000', 'Iteration: 975', 'Train loss: 0.444752', 'Train acc: 0.866667')
('Epoch: 195/2000', 'Iteration: 980', 'Train loss: 0.430450', 'Train acc: 0.863333')
('Epoch: 195/2000', 'Iteration: 980', 'Validation loss: 0.531019', 'Validation acc: 0.831667')
('Epoch: 196/2000', 'Iter

('Epoch: 250/2000', 'Iteration: 1255', 'Train loss: 0.327007', 'Train acc: 0.908333')
('Epoch: 251/2000', 'Iteration: 1260', 'Train loss: 0.343761', 'Train acc: 0.900000')
('Epoch: 251/2000', 'Iteration: 1260', 'Validation loss: 0.463211', 'Validation acc: 0.856667')
('Epoch: 252/2000', 'Iteration: 1265', 'Train loss: 0.318062', 'Train acc: 0.913333')
('Epoch: 253/2000', 'Iteration: 1270', 'Train loss: 0.315262', 'Train acc: 0.906667')
('Epoch: 253/2000', 'Iteration: 1270', 'Validation loss: 0.460071', 'Validation acc: 0.860000')
('Epoch: 254/2000', 'Iteration: 1275', 'Train loss: 0.308481', 'Train acc: 0.910000')
('Epoch: 255/2000', 'Iteration: 1280', 'Train loss: 0.338272', 'Train acc: 0.896667')
('Epoch: 255/2000', 'Iteration: 1280', 'Validation loss: 0.458967', 'Validation acc: 0.855000')
('Epoch: 256/2000', 'Iteration: 1285', 'Train loss: 0.319261', 'Train acc: 0.901667')
('Epoch: 257/2000', 'Iteration: 1290', 'Train loss: 0.332884', 'Train acc: 0.898333')
('Epoch: 257/2000', 'Ite

...

('Epoch: 1238/2000', 'Iteration: 6195', 'Train loss: 0.014971', 'Train acc: 0.998333')
('Epoch: 1239/2000', 'Iteration: 6200', 'Train loss: 0.020849', 'Train acc: 0.996667')
('Epoch: 1239/2000', 'Iteration: 6200', 'Validation loss: 0.419159', 'Validation acc: 0.920000')
('Epoch: 1240/2000', 'Iteration: 6205', 'Train loss: 0.019891', 'Train acc: 0.998333')
('Epoch: 1241/2000', 'Iteration: 6210', 'Train loss: 0.024244', 'Train acc: 0.996667')
('Epoch: 1241/2000', 'Iteration: 6210', 'Validation loss: 0.416366', 'Validation acc: 0.918333')
('Epoch: 1242/2000', 'Iteration: 6215', 'Train loss: 0.031358', 'Train acc: 0.990000')
('Epoch: 1243/2000', 'Iteration: 6220', 'Train loss: 0.024068', 'Train acc: 0.998333')
('Epoch: 1243/2000', 'Iteration: 6220', 'Validation loss: 0.415994', 'Validation acc: 0.923333')
('Epoch: 1244/2000', 'Iteration: 6225', 'Train loss: 0.020251', 'Train acc: 0.995000')
('Epoch: 1245/2000', 'Iteration: 6230', 'Train loss: 0.021164', 'Train acc: 0.995000')
('Epoch: 1245

('Epoch: 1299/2000', 'Iteration: 6500', 'Train loss: 0.014197', 'Train acc: 1.000000')
('Epoch: 1299/2000', 'Iteration: 6500', 'Validation loss: 0.423943', 'Validation acc: 0.928333')
('Epoch: 1300/2000', 'Iteration: 6505', 'Train loss: 0.019575', 'Train acc: 0.995000')
('Epoch: 1301/2000', 'Iteration: 6510', 'Train loss: 0.015731', 'Train acc: 0.998333')
('Epoch: 1301/2000', 'Iteration: 6510', 'Validation loss: 0.428158', 'Validation acc: 0.921667')
('Epoch: 1302/2000', 'Iteration: 6515', 'Train loss: 0.019946', 'Train acc: 0.996667')
('Epoch: 1303/2000', 'Iteration: 6520', 'Train loss: 0.021286', 'Train acc: 0.993333')
('Epoch: 1303/2000', 'Iteration: 6520', 'Validation loss: 0.421756', 'Validation acc: 0.921667')
('Epoch: 1304/2000', 'Iteration: 6525', 'Train loss: 0.010197', 'Train acc: 1.000000')
('Epoch: 1305/2000', 'Iteration: 6530', 'Train loss: 0.016757', 'Train acc: 0.998333')
('Epoch: 1305/2000', 'Iteration: 6530', 'Validation loss: 0.424661', 'Validation acc: 0.920000')
('E

('Epoch: 1360/2000', 'Iteration: 6805', 'Train loss: 0.018071', 'Train acc: 0.998333')
('Epoch: 1361/2000', 'Iteration: 6810', 'Train loss: 0.014884', 'Train acc: 0.998333')
('Epoch: 1361/2000', 'Iteration: 6810', 'Validation loss: 0.434434', 'Validation acc: 0.920000')
('Epoch: 1362/2000', 'Iteration: 6815', 'Train loss: 0.014224', 'Train acc: 1.000000')
('Epoch: 1363/2000', 'Iteration: 6820', 'Train loss: 0.023908', 'Train acc: 0.991667')
('Epoch: 1363/2000', 'Iteration: 6820', 'Validation loss: 0.427603', 'Validation acc: 0.925000')
('Epoch: 1364/2000', 'Iteration: 6825', 'Train loss: 0.012986', 'Train acc: 1.000000')
('Epoch: 1365/2000', 'Iteration: 6830', 'Train loss: 0.016240', 'Train acc: 0.998333')
('Epoch: 1365/2000', 'Iteration: 6830', 'Validation loss: 0.429810', 'Validation acc: 0.923333')
('Epoch: 1366/2000', 'Iteration: 6835', 'Train loss: 0.020926', 'Train acc: 0.993333')
('Epoch: 1367/2000', 'Iteration: 6840', 'Train loss: 0.018040', 'Train acc: 0.993333')
('Epoch: 1367

('Epoch: 1421/2000', 'Iteration: 7110', 'Train loss: 0.020627', 'Train acc: 0.995000')
('Epoch: 1421/2000', 'Iteration: 7110', 'Validation loss: 0.427820', 'Validation acc: 0.928333')
('Epoch: 1422/2000', 'Iteration: 7115', 'Train loss: 0.017034', 'Train acc: 0.995000')
('Epoch: 1423/2000', 'Iteration: 7120', 'Train loss: 0.009577', 'Train acc: 1.000000')
('Epoch: 1423/2000', 'Iteration: 7120', 'Validation loss: 0.426376', 'Validation acc: 0.923333')
('Epoch: 1424/2000', 'Iteration: 7125', 'Train loss: 0.021012', 'Train acc: 0.996667')
('Epoch: 1425/2000', 'Iteration: 7130', 'Train loss: 0.008864', 'Train acc: 1.000000')
('Epoch: 1425/2000', 'Iteration: 7130', 'Validation loss: 0.432451', 'Validation acc: 0.921667')
('Epoch: 1426/2000', 'Iteration: 7135', 'Train loss: 0.012171', 'Train acc: 0.998333')
('Epoch: 1427/2000', 'Iteration: 7140', 'Train loss: 0.011562', 'Train acc: 0.996667')
('Epoch: 1427/2000', 'Iteration: 7140', 'Validation loss: 0.431209', 'Validation acc: 0.920000')
('E

('Epoch: 1482/2000', 'Iteration: 7415', 'Train loss: 0.013653', 'Train acc: 0.998333')
('Epoch: 1483/2000', 'Iteration: 7420', 'Train loss: 0.016502', 'Train acc: 0.996667')
('Epoch: 1483/2000', 'Iteration: 7420', 'Validation loss: 0.440678', 'Validation acc: 0.920000')
('Epoch: 1484/2000', 'Iteration: 7425', 'Train loss: 0.007843', 'Train acc: 1.000000')
('Epoch: 1485/2000', 'Iteration: 7430', 'Train loss: 0.020111', 'Train acc: 0.995000')
('Epoch: 1485/2000', 'Iteration: 7430', 'Validation loss: 0.435465', 'Validation acc: 0.923333')
('Epoch: 1486/2000', 'Iteration: 7435', 'Train loss: 0.017163', 'Train acc: 0.993333')
('Epoch: 1487/2000', 'Iteration: 7440', 'Train loss: 0.014419', 'Train acc: 0.998333')
('Epoch: 1487/2000', 'Iteration: 7440', 'Validation loss: 0.428718', 'Validation acc: 0.925000')
('Epoch: 1488/2000', 'Iteration: 7445', 'Train loss: 0.010995', 'Train acc: 1.000000')
('Epoch: 1489/2000', 'Iteration: 7450', 'Train loss: 0.010853', 'Train acc: 1.000000')
('Epoch: 1489

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []
with graph.as_default():
    prediction=tf.argmax(logits,1)
with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    y_pred = []
    y_true = []
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        batch_acc, batch_y_pred = sess.run([accuracy, prediction], feed_dict=feed)
        y_pred.extend(batch_y_pred)
        y_true.extend(np.where(r==1)[0][0] for r in y_t )
        
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
#     print y_true
#     print y_pred
    sk_class_labels = [i for i in range(NUM_CLASS)]
    cnf_matrix = confusion_matrix(y_true, y_pred)
    print precision_recall_fscore_support(y_true, y_pred, average=None, labels=sk_class_labels)
    print 'Accuracy:', accuracy_score(y_true, y_pred)
    print 'F1 score:', f1_score(y_true, y_pred, average='micro')
    print 'Recall:', recall_score(y_true, y_pred, average='micro')
    print 'Precision:', precision_score(y_true, y_pred, average='micro')
    print '\n clasification report:\n', classification_report(y_true,y_pred)
    print '\n confussion matrix:\n',cnf_matrix
    fig, ax = plt.subplots() 
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 20
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plot_confusion_matrix(cnf_matrix, class_label_vn, title='Confusion matrix')

    plt.show()